# How to build an AMBER-compatible host-guest complex using the SMIRNOFF force field starting with SYBYL-formatted `mol2` files

In [65]:
import parmed as pmd
import subprocess as sp
import numpy as np

from openeye import oechem
from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

## Step 0: generate SYBYL-formatted `mol2` files

Starting with files containing AM1-BCC charges and GAFF v1.7 Lennard-Jones and bonded parameters:

```
antechamber -i bcd.mol2 -fi mol2 -o bcd-sybyl.mol2 -fo mol2 -at sybyl
antechamber -i ben.mol2 -fi mol2 -o ben-sybyl.mol2 -fo mol2 -at sybyl -dr n 
# Disable `acdoctor` to handle carboxylate group
```

In [66]:
path = './bcd-ben/'
bcd = path + 'bcd-sybyl.mol2'
ben = path + 'ben-sybyl.mol2'

In [67]:
!head -n 10 './bcd-ben/bcd.mol2' './bcd-ben/bcd-sybyl.mol2' 

==> ./bcd-ben/bcd.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj generated mol2 file.
  147   154     7     0     0
SMALL
USER_CHARGES


@<TRIPOS>ATOM
      1 C1         16.6428   14.6231   33.9447 c3         1 MGO      0.304529
      2 H1         15.6508   14.1481   33.8157 h2         1 MGO      0.090782

==> ./bcd-ben/bcd-sybyl.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj
  147   154     1     0     0
SMALL
No Charge or Current Charge


@<TRIPOS>ATOM
      1 C1          16.6430    14.6230    33.9450 C.3        1 MGO       0.304529
      2 H1          15.6510    14.1480    33.8160 H          1 MGO       0.090782


In [68]:
!head -n 10 './bcd-ben/ben.mol2' './bcd-ben/ben-sybyl.mol2'

==> ./bcd-ben/ben.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj generated mol2 file.
   14    14     1     0     0
SMALL
USER_CHARGES


@<TRIPOS>ATOM
      1 C1         19.6968   19.9941   32.6877 ca         1 BEN     -0.161879
      2 H1         19.6968   19.9941   31.5977 ha         1 BEN      0.097073

==> ./bcd-ben/ben-sybyl.mol2 <==
@<TRIPOS>MOLECULE
Cpptraj
   14    14     1     0     0
SMALL
No Charge or Current Charge


@<TRIPOS>ATOM
      1 C1          19.6970    19.9940    32.6880 C.ar       1 BEN      -0.161879
      2 H1          19.6970    19.9940    31.5980 H          1 BEN       0.097073


## Step 1: load molecules in an `oechem.OEMol`

In [69]:
mols = []
ifs = oechem.oemolistream(bcd)
for mol in ifs.GetOEGraphMols():
    mols.append(oechem.OEMol(mol))

## Step 1.5: since `bcd` is a cyclic molecule, let's write a version without connectivity and then have OpenEye perceive connectivity based on [this issue](https://github.com/openforcefield/openforcefield/issues/66)

In [70]:
ofs = oechem.oemolostream()
ofs.open(path + 'bcd-no-conect.xyz')
flavor = oechem.OEOFlavor_Generic_Default
ofs.SetFlavor(oechem.OEFormat_XYZ, flavor)
oechem.OEWriteXYZFile(ofs, mols[0])
ofs.close()

In [71]:
ifs = oechem.oemolistream(path + 'bcd-no-conect.xyz')
for mol in ifs.GetOEGraphMols():
    oechem.OEDetermineConnectivity(mol)
    oechem.OEPerceiveBondOrders(mol)
    # Replace existing cyclodextrin
    mols[0] = oechem.OEMol(mol)

In [72]:
ifs = oechem.oemolistream(ben)
for mol in ifs.GetOEGraphMols():
    mols.append(oechem.OEMol(mol))

## Step 2: build reference molecules for water and ions

In [73]:
smiles = ['[Na+]', '[Cl-]', 'O']
for molecule in smiles:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, molecule)
    for atom in mol.GetAtoms():
        atom.SetPartialCharge(atom.GetFormalCharge())
    oechem.OEAddExplicitHydrogens(mol)
    oechem.OETriposAtomNames(mol)
    mols.append(mol)

## Step 3: read in fully solvated system with host, guest, water, and ions

### Step 3.1: create a PDB of the system with CONECT records
This requires using the "original" `mol2` files with GAFF atom names.

In [74]:
solvated_pdb = 'bcd-ben.pdb'
amber_prmtop = 'solvated.prmtop'
pdb_with_conect = 'bcd-ben-conect.pdb'
cpptraj_input = 'bcd-ben-conect.in'

cpptraj = \
'''
parm {}
trajin {}
trajout {} conect
'''.format(amber_prmtop, solvated_pdb, pdb_with_conect)

In [75]:
with open(path + cpptraj_input, 'w') as file:
    file.write(cpptraj)

In [76]:
cpptraj_output = sp.check_output(['cpptraj', '-i', cpptraj_input], cwd=path)

### Step 3.2: prune the CONECT records that correspond specifically to water molecules

In [77]:
first_water = sp.check_output(['grep', '-m 1', 'WAT', pdb_with_conect], cwd=path).decode("utf-8") 

In [78]:
first_water_residue = int(float(first_water.split()[1]))
print('First water residue = {}'.format(first_water_residue))

First water residue = 175


In [79]:
line_of_first_conect_to_delete = sp.check_output(['egrep', '-n', 'CONECT [ ]* {}'.format(str(first_water_residue)), 
                                                  pdb_with_conect], cwd=path).decode("utf-8")

In [81]:
line_to_delete_from = int(float(line_of_first_conect_to_delete.split(':')[0]))
print('Found first water CONECT entry at line = {}'.format(line_to_delete_from))

Found first water CONECT entry at line = 9192


In [82]:
truncated_file = sp.check_output(['awk', 'NR < {}'.format(line_to_delete_from), pdb_with_conect], cwd=path).decode("utf-8")

In [87]:
file = open(path + 'tmp.pdb', 'w')
file.write(truncated_file)
file.write("END")
file.close()
p = sp.check_output(['mv', 'tmp.pdb', '{}'.format(pdb_with_conect)], cwd=path)

b''

In [88]:
pdb = PDBFixer(path + pdb_with_conect)

In [90]:
# residues = [r.name for r in pdb.topology.residues()]
# print(residues)

In [92]:
# names = [a.name for a in pdb.topology.atoms()]
# print(names)

## Step 4: create a ParmEd structure

In [101]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml', 'forcefield/tip3p.ffxml') 
system = ff.createSystem(pdb.topology, 
                         mols)
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)

## Step 5: save the ParmEd structure as a `.prmtop` file

In [102]:
structure = pmd.openmm.topsystem.load_topology(pdb.topology, system, pdb.positions)

In [104]:
structure.save(path + 'bcd-ben-smirnoff.prmtop')

AttributeError: 'NoneType' object has no attribute 'used'